### Setup domain:

notebooks/Experimental/Ishan/AA/setup-data.ipynb

In [1]:
import syft as sy

/home/e/anaconda3/envs/Hagrid/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
domain = sy.login(email="info@openmined.org", password="changethis", port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into adp... done!


In [3]:
domain.datasets

Idx,Name,Description,Assets,Id
[0],BreastCancerDataset,Invasive Ductal Carcinoma (IDC) is the most common subtype of all breast cancers. The modified dataset consisted of 162 whole mount slide images of Breast Cancer (BCa) specimens scanned at 40x. Patches of size 50 x 50 were extracted from the original image. The labels 0 is non-IDC and 1 is IDC.,"[""train_images""] -> [""train_labels""] ->",be953051-3f0b-4ad7-844a-0ca17455f5e5
[1],BreastCancerDataset,Invasive Ductal Carcinoma (IDC) is the most common subtype of all breast cancers. The modified dataset consisted of 162 whole mount slide images of Breast Cancer (BCa) specimens scanned at 40x. Patches of size 50 x 50 were extracted from the original image. The labels 0 is non-IDC and 1 is IDC.,"[""train_images""] -> [""train_labels""] ->",4af02b1b-1304-4bde-b4b4-1e16378edd92


In [4]:
X_train = domain.datasets[-1]["train_images"][:2]
y_train = domain.datasets[-1]["train_labels"][:2]

In [5]:
input_shape = X_train[:2].public_shape

In [6]:
input_shape

(2, 3, 50, 50)

In [7]:
from syft import nn

In [8]:
model = nn.Model()

# Layer 1
model.add(nn.Convolution(nb_filter=32, filter_size=3, padding=2, input_shape=input_shape))
model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
model.add(nn.MaxPool(pool_size=2, stride=2))

# Layer 2
model.add(nn.Convolution(nb_filter=64, filter_size=3, padding=2))
model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
model.add(nn.MaxPool(pool_size=2, stride=2))

# # Layer 3
# model.add(nn.Convolution(nb_filter=128, filter_size=3, padding=2))
# model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
# model.add(nn.MaxPool(pool_size=2, stride=2))

# # Layer 4
# model.add(nn.Convolution(nb_filter=256, filter_size=3, padding=2))
# model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
# model.add(nn.MaxPool(pool_size=2, stride=2))

# Layer 5
model.add(nn.AvgPool(5))

# Layer 6
model.add(nn.Flatten())

# Layer 7
model.add(nn.Linear(2, 512))

In [9]:
model.initialize_weights()

In [10]:
model.layers

In [11]:
model_ptr = model.send(domain,send_to_blob_storage=False)

In [12]:
x_batch, y_batch = X_train[:2], y_train[:2]

In [13]:
import time
from tqdm import tqdm
n_epochs = 1
batch_size = 2
for epoch in tqdm(range(n_epochs)):
    print(f"Epoch: {epoch}")
    total_images = X_train.public_shape[0]
    start_idx = 0
    while(start_idx < total_images):
        end_idx = start_idx+batch_size
        x_batch = X_train[start_idx:end_idx]
        y_batch = y_train[start_idx:end_idx]
        run_status = model_ptr.step(x_batch, y_batch)
        while(not run_status.exists):
            time.sleep(10)
        print("Epoch completed.")
        start_idx += batch_size

  0%|                                                                | 0/1 [00:00<?, ?it/s]

Epoch: 0


100%|███████████████████████████████████████████████████████| 1/1 [04:00<00:00, 240.71s/it]

Epoch completed.


In [14]:
# Need to implement download_weights
weights = model_ptr.publish(sigma=1000)

In [15]:
while not weights.exists:
    time.sleep(5)

In [16]:
res = weights.get_copy()

In [20]:
second_model = nn.Model()

# Layer 1
second_model.add(nn.Convolution(nb_filter=32, filter_size=3, padding=2, input_shape=input_shape))
second_model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
second_model.add(nn.MaxPool(pool_size=2, stride=2))

# Layer 2
second_model.add(nn.Convolution(nb_filter=64, filter_size=3, padding=2))
second_model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
second_model.add(nn.MaxPool(pool_size=2, stride=2))

# Layer 5
second_model.add(nn.AvgPool(5))

# Layer 6
second_model.add(nn.Flatten())

# Layer 7
second_model.add(nn.Linear(2, 512))

second_model.initialize_weights()

In [63]:
def replace_weights(published_weights: dict) -> None:
    """
    For when you want to use published weights downloaded from a domain
    
    """
    layer_keys = list(published_weights.keys())
    print(layer_keys)
    for i, layer in enumerate(second_model.layers):
        new_weights = published_weights[layer_keys[i]]
        if len(new_weights) == 0:
            continue
        elif len(new_weights) == 2:
            if isinstance(layer, nn.Convolution):
                layer.W = new_weights[0]
                layer.b = new_weights[1].flatten()[:layer.b.shape[0]]
            elif isinstance(layer, nn.BatchNorm):
                layer.beta = new_weights[0].flatten()[:layer.beta.shape[0]]  # TO BYPASS OPTIMIZER ERROR FOR NOW
                layer.gamma = new_weights[1].flatten()[:layer.gamma.shape[0]]
            elif isinstance(layer, nn.Linear):
                layer.W = new_weights[0]
                layer.b = new_weights[1]
            else:
                raise NotImplementedError
        else:
            print(i, len(new_weights))
            raise NotImplementedError


In [64]:
replace_weights(res)

['Convolution0', 'BatchNorm1', 'MaxPool2', 'Convolution3', 'BatchNorm4', 'MaxPool5', 'AvgPool6', 'Flatten7', 'Linear8']


In [69]:
second_model.layers[-1].b.shape

(2,)

In [70]:
model.layers[-1].b.shape

(2,)

In [72]:
domain2 = sy.login(email="info@openmined.org", password="changethis", port=8082)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into vigilant_sutskever... done!


In [73]:
domain2.datasets

Idx,Name,Description,Assets,Id
[0],BreastCancerDataset,Invasive Ductal Carcinoma (IDC) is the most common subtype of all breast cancers. The modified dataset consisted of 162 whole mount slide images of Breast Cancer (BCa) specimens scanned at 40x. Patches of size 50 x 50 were extracted from the original image. The labels 0 is non-IDC and 1 is IDC.,"[""train_images""] -> [""train_labels""] ->",dde3bf76-9109-4405-baf8-d9278d6db316


In [76]:
model_ptr2 = model.send(domain2,send_to_blob_storage=False)

In [78]:
X_train2 = domain2.datasets[-1]["train_images"][:2]
y_train2 = domain2.datasets[-1]["train_labels"][:2]

In [79]:
input_shape2 = X_train2[:2].public_shape

In [80]:
n_epochs = 1
batch_size = 2

for epoch in tqdm(range(n_epochs)):
    print(f"Epoch: {epoch}")
    total_images = X_train2.public_shape[0]
    start_idx = 0
    while(start_idx < total_images):
        end_idx = start_idx+batch_size
        x_batch = X_train2[start_idx:end_idx]
        y_batch = y_train2[start_idx:end_idx]
        run_status = model_ptr2.step(x_batch, y_batch)
        while(not run_status.exists):
            time.sleep(10)
        print("Epoch completed.")
        start_idx += batch_size

  0%|                                                                                                                                                           | 0/1 [00:00<?, ?it/s]

Epoch: 0


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [04:00<00:00, 240.71s/it]

Epoch completed.


In [81]:
# Need to implement download_weights
weights2 = model_ptr2.publish(sigma=1000)

In [82]:
while not weights2.exists:
    time.sleep(5)

In [83]:
res2 = weights2.get_copy()

In [84]:
res2

{'Convolution0': [array([[[[-9.80039894e+01,  1.02768522e+03, -6.16149072e+02],
         [ 1.31866410e+03, -9.84277856e+02,  3.03234790e+02],
         [-2.32951540e+02,  2.50028409e+02, -1.28778312e+03]],

        [[ 1.71814019e+03,  9.73240483e+02,  1.21909587e+02],
         [ 5.47957397e+02,  3.30544324e+02, -1.20478664e+03],
         [-6.07045887e+02,  2.52798406e+02, -4.00559244e+02]],

        [[ 2.27142343e+02,  1.50426947e+03, -4.93999782e+02],
         [-1.99857420e+02, -4.70274618e+02,  1.01574877e+03],
         [-2.29752755e+02,  1.92462759e+03,  6.71538265e+02]]],


       [[[ 9.87191880e+01,  6.52084743e+02, -1.53557676e+03],
         [ 7.01932745e+02, -2.41002904e+03,  1.54512265e+03],
         [-3.96522117e+02, -9.58510879e+02,  6.61622891e+02]],

        [[-1.69504237e+02, -1.82033964e+03, -3.32899571e+02],
         [ 3.00360887e+02, -1.75424720e+03,  1.03955599e+03],
         [ 1.43055467e+03, -1.08939868e+03, -5.44231873e+02]],

        [[ 1.59026591e+03,  5.49248531e+